## BIOINFORMATICS THESIS: MULTIMODAL NEURAL NETWORK

# CELL LINE: A549
# EMBRACENET

da fare 
- sistema selezione augmentation in embracenet
- crea grafici AUPRC reti

In [1]:
import pandas as pd 
import numpy as np
import os

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict, OrderedDict
import pickle

import sqlite3
from sqlalchemy import create_engine

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
#!rm BIOINF_optuna_tuning.db

In [3]:
# create a database to store optuna studies with sqlite backendù
#engine = create_engine('sqlite:///BIOINF_optuna_tuning.db')

In [4]:
from BIOINF_tesi.data_pipe import CELL_LINES, TASKS

In [5]:
cell_line = CELL_LINES[0]
cell_line

'A549'

---

In [6]:
from BIOINF_tesi.data_pipe import Build_DataLoader_Pipeline
from BIOINF_tesi.models import EmbraceNetMultimodal, ConcatNetMultimodal
from BIOINF_tesi.models.utils import Kfold_CV_Multimodal

In [7]:
with open ('results_dict.pickle', 'rb') as fin:
    results_dict = pickle.load(fin)
    results_dict = defaultdict(lambda: defaultdict(dict), results_dict)

---

In [8]:
from BIOINF_tesi.models.utils import select_augmented_models
results_dict = select_augmented_models(results_dict, verbose=True)


A549
active_E_vs_inactive_E
pvalue: 0.5126907602619233
Best augmentation method: double

A549
active_EP_vs_inactive_rest
pvalue: 0.5126907602619233
Best augmentation method: double

K562
active_E_vs_inactive_E
pvalue: 0.8272593465627113
Best augmentation method: double

MCF7
active_E_vs_inactive_E
pvalue: 0.8272593465627113
Best augmentation method: double

H1
active_E_vs_inactive_E
pvalue: 0.8272593465627113
Best augmentation method: double


In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

## 1) ACTIVE ENHANCERS vs INACTIVE ENHANCERS

In [9]:
task = TASKS[0]

In [10]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [11]:
model=EmbraceNetMultimodal

In [12]:
# IMBALANCED 
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-23 11:17:37,825] Using an existing study with name 'A549_active_E_vs_inactive_E_EmbraceNetMultimodal_1' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)
/opt/anaconda3/lib/python3.8/site-packages/timm/optim/nadam.py:69: UserWarning: This overload of add is deprecated:
	add(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1025.)
  grad = grad.add(group['weight_decay'], p.data)


EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


...

EarlyStopping counter: 2 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-23 18:29:11,084] Trial 2 finished with value: 0.11884563542743685 and parameters: {'FFNN_n_layers': 3, 'FFNN_n_units_l0': 32, 'FFNN_dropout_l0': 0.0, 'FFNN_n_units_l1': 16, 'FFNN_dropout_l1': 0.4, 'FFNN_n_units_l2': 16, 'FFNN_dropout_l2': 0.0, 'CNN_n_layers': 3, 'CNN_out_channels_l0': 32, 'CNN_kernel_size_l0': 11, 'CNN_dropout_l0': 0.4, 'CNN_out_channels_l1': 32, 'CNN_kernel_size_l1': 15, 'CNN_dropout_l1': 0.4, 'CNN_out_channels_l2': 64, 'CNN_kernel_size_l2': 15, 'CNN_dropout_l2': 0, 'EMBRACENET_embracement_size': 768, 'n_post_layers': 0, 'selection_probabilities_FFNN': 0.4410524982384807, 'optimizer': 'RMSprop', 'lr': 0.009305526103875358, 'weight_decay': 0.0004895293285311643}. Best is trial 2 with value: 0.11884563542743685.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.11884563542743685
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 15
    CNN_kernel_size_l2: 15
    CNN_n_layers: 3
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 32
    CNN_out_channels_l2: 64
    EMBRACENET_embracement_size: 768
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.0
    FFNN_n_layers: 3
    FFNN_n_units_l0: 32
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 16
    lr: 0.009305526103875358
    n_post_layers: 0
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.4410524982384807
    weight_decay: 0.0004895293285311643

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.1346804268038381



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.09301


In [13]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [14]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [15]:
model=ConcatNetMultimodal

In [16]:
# IMBALANCED 
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-23 18:47:36,079] A new study created in RDB with name: A549_active_E_vs_inactive_E_ConcatNetMultimodal_1


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-23 19:08:46,409] Trial 0 finished with value: 0.05440392354124748 and parameters: {'FFNN_n_layers': 2, 'FFNN_n_units_l0': 64, 'FFNN_dropout_l0': 0.3, 'FFNN_n_units_l1': 64, 'FFNN_dropout_l1': 0.0, 'CNN_n_layers': 3, 'CNN_out_channels_l0': 16, 'CNN_kernel_size_l0': 11, 'CNN_dropout_l0': 0.4, 'CNN_out_channels_l1': 32, 'CNN_kernel_size_l1': 11, 'CNN_dropout_l1': 0.5, 'CNN_out_channels_l2': 256, 'CNN_kernel_size_l2': 15, 'CNN_dropout_l2': 0.4, 'CONCATNET_n_post_layers': 1, 'CONCATNET_n_units_l0': 1024, 'CONCATNET_dropout_l0': 0.3, 'optimizer': 'Nadam', 'lr': 0.00014039850956131414, 'weight_decay': 0.000672647926688101}. Best is trial 0 with value: 0.05440392354124748.


...

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5


KeyboardInterrupt: 

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 2) ACTIVE PROMOTERS vs INACTIVE PROMOTERS

In [17]:
task = TASKS[1]
task

'active_P_vs_inactive_P'

In [18]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [19]:
model=EmbraceNetMultimodal

In [20]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-23 21:33:52,917] A new study created in RDB with name: A549_active_P_vs_inactive_P_EmbraceNetMultimodal_1


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-23 22:20:20,534] Trial 0 finished with value: 0.09776797205368636 and parameters: {'FFNN_n_layers': 3, 'FFNN_n_units_l0': 128, 'FFNN_dropout_l0': 0.3, 'FFNN_n_units_l1': 32, 'FFNN_dropout_l1': 0.2, 'FFNN_n_units_l2': 4, 'FFNN_dropout_l2': 0.0, 'CNN_n_layers': 4, 'CNN_out_channels_l0': 64, 'CNN_kernel_size_l0': 5, 'CNN_dropout_l0': 0.4, 'CNN_out_channels_l1': 32, 'CNN_kernel_size_l1': 15, 'CNN_dropout_l1': 0.4, 'CNN_out_channels_l2': 96, 'CNN_kernel_size_l2': 11, 'CNN_dropout_l2': 0, 'CNN_out_channels_l3': 512, 'CNN_kernel_size_l3': 15, 'CNN_dropout_l3': 0, 'EMBRACENET_embracement_size': 512, 'n_post_layers': 0, 'selection_probabilities_FFNN': 0.3219476258910442, 'optimizer': 'Adam', 'lr': 1.4308001681419734e-05, 'weight_decay': 0.0050545247865213345}. Best is trial 0 with value: 0.09776797205368636.


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-24 08:11:39,371] Trial 2 finished with value: 0.10209229637801069 and parameters: {'FFNN_n_layers': 1, 'FFNN_n_units_l0': 64, 'FFNN_dropout_l0': 0.3, 'CNN_n_layers': 4, 'CNN_out_channels_l0': 32, 'CNN_kernel_size_l0': 11, 'CNN_dropout_l0': 0, 'CNN_out_channels_l1': 32, 'CNN_kernel_size_l1': 5, 'CNN_dropout_l1': 0.5, 'CNN_out_channels_l2': 256, 'CNN_kernel_size_l2': 5, 'CNN_dropout_l2': 0.5, 'CNN_out_channels_l3': 128, 'CNN_kernel_size_l3': 15, 'CNN_dropout_l3': 0, 'EMBRACENET_embracement_size': 1024, 'n_post_layers': 0, 'selection_probabilities_FFNN': 0.08722229651566682, 'optimizer': 'Adam', 'lr': 2.8318771934885095e-05, 'weight_decay': 0.0002669325513188872}. Best is trial 1 with value: 0.12358197160651382.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.12358197160651382
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.4
    CNN_dropout_l2: 0.4
    CNN_dropout_l3: 0
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 11
    CNN_kernel_size_l2: 5
    CNN_kernel_size_l3: 15
    CNN_n_layers: 4
    CNN_out_channels_l0: 16
    CNN_out_channels_l1: 96
    CNN_out_channels_l2: 96
    CNN_out_channels_l3: 512
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 512
    FFNN_dropout_l0: 0.0
    FFNN_dropout_l1: 0.2
    FFNN_dropout_l2: 0.5
    FFNN_dropout_l3: 0.0
    FFNN_n_layers: 4
    FFNN_n_units_l0: 256
    FFNN_n_units_l1: 32
    FFNN_n_units_l2: 4
    FFNN_n_units_l3: 4
    lr: 0.05666033129014694
    n_post_layers: 1
    optimizer: Adam
    selection_probabilities_FFNN: 0.7076055771501554


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.1034537376859185



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.18169


In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [ ]:
model=ConcatNetMultimodal

In [ ]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 3) ACTIVE ENHANCERS vs ACTIVE PROMOTERS

In [21]:
task = TASKS[2]
task

'active_E_vs_active_P'

In [22]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [23]:
model=EmbraceNetMultimodal

In [24]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-24 09:45:57,734] A new study created in RDB with name: A549_active_E_vs_active_P_EmbraceNetMultimodal_1


>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-24 09:53:26,060] Trial 0 finished with value: 0.6768720385991267 and parameters: {'FFNN_n_layers': 1, 'FFNN_n_units_l0': 256, 'FFNN_dropout_l0': 0.2, 'CNN_n_layers': 2, 'CNN_out_channels_l0': 16, 'CNN_kernel_size_l0': 5, 'CNN_dropout_l0': 0.3, 'CNN_out_channels_l1': 64, 'CNN_kernel_size_l1': 5, 'CNN_dropout_l1': 0.5, 'EMBRACENET_embracement_size': 1024, 'n_post_layers': 2, 'EMBRACENET_n_units_l0': 64, 'EMBRACENET_dropout_l0': 0.5, 'EMBRACENET_n_units_l1': 16, 'EMBRACENET_dropout_l1': 0.3, 'selection_probabilities_FFNN': 0.935192174728515, 'optimizer': 'Nadam', 'lr': 0.0007885779244108592, 'weight_decay': 0.002380136234668125}. Best is trial 0 with value: 0.6768720385991267.


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-24 12:14:43,677] Trial 2 finished with value: 0.4439038489821603 and parameters: {'FFNN_n_layers': 4, 'FFNN_n_units_l0': 32, 'FFNN_dropout_l0': 0.3, 'FFNN_n_units_l1': 64, 'FFNN_dropout_l1': 0.4, 'FFNN_n_units_l2': 32, 'FFNN_dropout_l2': 0.4, 'FFNN_n_units_l3': 4, 'FFNN_dropout_l3': 0.0, 'CNN_n_layers': 3, 'CNN_out_channels_l0': 64, 'CNN_kernel_size_l0': 15, 'CNN_dropout_l0': 0.3, 'CNN_out_channels_l1': 64, 'CNN_kernel_size_l1': 11, 'CNN_dropout_l1': 0.5, 'CNN_out_channels_l2': 64, 'CNN_kernel_size_l2': 11, 'CNN_dropout_l2': 0.4, 'EMBRACENET_embracement_size': 512, 'n_post_layers': 1, 'EMBRACENET_n_units_l0': 64, 'EMBRACENET_dropout_l0': 0.2, 'selection_probabilities_FFNN': 0.8253492289630943, 'optimizer': 'Nadam', 'lr': 4.2581699078399265e-05, 'weight_decay': 0.011901731153611136}. Best is trial 0 with value: 0.6868036254118559.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.6868036254118559
  Params: 
    CNN_dropout_l0: 0.4
    CNN_kernel_size_l0: 15
    CNN_n_layers: 1
    CNN_out_channels_l0: 16
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_embracement_size: 768
    EMBRACENET_n_units_l0: 64
    FFNN_dropout_l0: 0.3
    FFNN_dropout_l1: 0.4
    FFNN_n_layers: 2
    FFNN_n_units_l0: 64
    FFNN_n_units_l1: 32
    lr: 0.00037105954310415515
    n_post_layers: 1
    optimizer: Adam
    selection_probabilities_FFNN: 0.07911471687076432
    weight_decay: 0.00021222132767789495

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.6755977372853926



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.68973


In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [ ]:
model=ConcatNetMultimodal

In [ ]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 4) INACTIVE ENHANCERS vs INACTIVE PROMOTERS

In [25]:
task = TASKS[3]
task

'inactive_E_vs_inactive_P'

In [26]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [27]:
model=EmbraceNetMultimodal

In [28]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-24 12:28:25,989] A new study created in RDB with name: A549_inactive_E_vs_inactive_P_EmbraceNetMultimodal_1


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-24 13:52:54,680] Trial 0 finished with value: 0.5867011514052644 and parameters: {'FFNN_n_layers': 3, 'FFNN_n_units_l0': 128, 'FFNN_dropout_l0': 0.4, 'FFNN_n_units_l1': 128, 'FFNN_dropout_l1': 0.4, 'FFNN_n_units_l2': 32, 'FFNN_dropout_l2': 0.0, 'CNN_n_layers': 2, 'CNN_out_channels_l0': 32, 'CNN_kernel_size_l0': 15, 'CNN_dropout_l0': 0.4, 'CNN_out_channels_l1': 96, 'CNN_kernel_size_l1': 11, 'CNN_dropout_l1': 0, 'EMBRACENET_embracement_size': 1024, 'n_post_layers': 0, 'selection_probabilities_FFNN': 0.25537997363122, 'optimizer': 'Adam', 'lr': 0.0006083751786931687, 'weight_decay': 0.0003558635367740515}. Best is trial 0 with value: 0.5867011514052644.


...

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5


[I 2021-10-25 03:52:11,491] Trial 2 finished with value: 0.5755169202394119 and parameters: {'FFNN_n_layers': 1, 'FFNN_n_units_l0': 256, 'FFNN_dropout_l0': 0.2, 'CNN_n_layers': 2, 'CNN_out_channels_l0': 64, 'CNN_kernel_size_l0': 11, 'CNN_dropout_l0': 0.2, 'CNN_out_channels_l1': 32, 'CNN_kernel_size_l1': 5, 'CNN_dropout_l1': 0.5, 'EMBRACENET_embracement_size': 512, 'n_post_layers': 0, 'selection_probabilities_FFNN': 0.9797959194457883, 'optimizer': 'Adam', 'lr': 0.010411917421571734, 'weight_decay': 0.00042456566871203227}. Best is trial 2 with value: 0.5755169202394119.


EarlyStopping counter: 5 out of 5
Early stopping the training
Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5755169202394119
  Params: 
    CNN_dropout_l0: 0.2
    CNN_dropout_l1: 0.5
    CNN_kernel_size_l0: 11
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 64
    CNN_out_channels_l1: 32
    EMBRACENET_embracement_size: 512
    FFNN_dropout_l0: 0.2
    FFNN_n_layers: 1
    FFNN_n_units_l0: 256
    lr: 0.010411917421571734
    n_post_layers: 0
    optimizer: Adam
    selection_probabilities_FFNN: 0.9797959194457883
    weight_decay: 0.00042456566871203227

===============> MODEL TESTING


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 1 out of 5
EarlyStopping counter: 2 out of 5
EarlyStopping counter: 3 out of 5
EarlyStopping counter: 4 out of 5
EarlyStopping counter: 5 out of 5
Early stopping the training
AUPRC test score: 0.562809515801858



3-FOLD CROSS-VALIDATION AUPRC TEST SCORE: 0.57851


In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [ ]:
model=ConcatNetMultimodal

In [ ]:
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

---

## 5) ACTIVE ENHANCERS + ACTIVE PROMOTERS vs INACTIVE REST

In [8]:
task = TASKS[4]
task

'active_EP_vs_inactive_rest'

In [9]:
pipe_data_load = Build_DataLoader_Pipeline(path_name=f'{task}.pickle')

Data Preprocessing Done!


---
### EMBRACENET

In [10]:
model=EmbraceNetMultimodal

In [ ]:
# IMBALANCED
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

>>> ITERATION N. 1

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 13:36:46,796] Using an existing study with name 'A549_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1' instead of creating a new one.
/opt/anaconda3/lib/python3.8/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.1504632861722721
  Params: 
    CNN_dropout_l0: 0.4
    CNN_dropout_l1: 0.4
    CNN_kernel_size_l0: 5
    CNN_kernel_size_l1: 5
    CNN_n_layers: 2
    CNN_out_channels_l0: 32
    CNN_out_channels_l1: 64
    EMBRACENET_dropout_l0: 0.0
    EMBRACENET_dropout_l1: 0.5
    EMBRACENET_embracement_size: 1024
    EMBRACENET_n_units_l0: 256
    EMBRACENET_n_units_l1: 16
    FFNN_dropout_l0: 0.4
    FFNN_dropout_l1: 0.4
    FFNN_dropout_l2: 0.5
    FFNN_dropout_l3: 0.5
    FFNN_n_layers: 4
    FFNN_n_units_l0: 128
    FFNN_n_units_l1: 16
    FFNN_n_units_l2: 32
    FFNN_n_units_l3: 32
    lr: 0.0053033183119394894
    n_post_layers: 2
    optimizer: RMSprop
    selection_probabilities_FFNN: 0.5420169718993605
    weight_decay: 0.0021288733188140397

===============> MODEL TESTING
AUPRC test score: 0.1124866688147842


>>> ITERATION N. 2

===============> HYPER

/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 13:36:52,523] Using an existing study with name 'A549_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_2' instead of creating a new one.


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

...

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 4
EarlyStopping counter: 2 out of 4
EarlyStopping counter: 3 out of 4
EarlyStopping counter: 1 out of 4
EarlyStopping counter: 2 out of 4
EarlyStopping counter: 3 out of 4
EarlyStopping counter: 4 out of 4
Early stopping the training
AUPRC test score: 0.2038105141769414


>>> ITERATION N. 3

===============> HYPERPARAMETERS TUNING


/Users/Niki/Prediction-of-Active-and-Inactive-Regulatory-Regions-with-Embracenet-Multimodal-Neural-Network-/BIOINF_tesi/models/utils/training_models_multimodal.py:425: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  pruner=optuna.pruners.PatientPruner(optuna.pruners.MedianPruner(), patience=2),
[I 2021-10-25 19:39:45,611] A new study created in RDB with name: A549_active_EP_vs_inactive_rest_EmbraceNetMultimodal_1_2_3


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

EarlyStopping counter: 1 out of 4
EarlyStopping counter: 2 out of 4


In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

### CONCATNET

In [ ]:
model=ConcatNetMultimodal

In [ ]:
# IMBALANCED
kf_CV = Kfold_CV_Multimodal()

kf_CV(build_dataloader_pipeline = pipe_data_load,
                num_epochs = 100,
                n_folds=3,
                cell_line=cell_line,
                augmentation=False,
                model = model,
                device = device,
                task=task,
                sampler = 'TPE',
                study_name = f'{cell_line}_{task}_{model.__name__}',
                test_model_path = f'{cell_line}_{task}_{model.__name__}_TEST')

In [ ]:
results_dict[cell_line][task][model.__name__] = kf_CV.scores_dict

In [ ]:
with open ('results_dict.pickle', 'wb') as fout:
    pickle.dump(OrderedDict(results_dict), fout)

In [ ]:
from BIOINF_tesi.models.utils import plot_scores
plot_scores(cells=cell_line, models=['EmbraceNetMultimodal', 'ConcatNetMultimodal' ])

## not freezed layers + NO AUGMENTATION

In [ ]:
from BIOINF_tesi.models.utils import plot_scores
plot_scores(cells=cell_line, models=['EmbraceNetMultimodal'])

## not freezed layers + AUGMENTATION

In [ ]:
from BIOINF_tesi.models.utils import plot_scores
plot_scores(cells=cell_line, models=['EmbraceNetMultimodal'])

## unimodal

In [ ]:
plot_scores(cells=cell_line, models=['CNN','FFNN'])